In [ ]:
import pandas as pd

# 读取数据文件
file_path = '/mnt/d/Research/PHD/DLEPS/data/compoundinfo_beta.txt'
data = pd.read_csv(file_path, sep='\t')

# 统计总行数
total_rows = data.shape[0]

# 统计canonical_smiles列中不重复数据的数量
unique_smiles = data['canonical_smiles'].nunique()

# 统计每组canonical_smiles的条数
smiles_counts = data['canonical_smiles'].value_counts()

# 输出结果
print(f"总行数: {total_rows}")
print(f"canonical_smiles列中不重复数据的数量: {unique_smiles}")
print(f"每组数据的条数:")
print(smiles_counts)

# 从`LINCS_small_molecules.tsv`中根据`GSE92742_Broad_LINCS_sig_info.txt`里面的BRD标识符匹配化合物SMILES
## 步骤一：准备数据

1. **加载数据**：

   - 从`GSE92742_Broad_LINCS_sig_info.txt`文件加载信号信息数据到`sig_info_df`。
   - 从`LINCS_small_molecules.tsv`文件加载小分子数据到`small_molecules_df`。

## 步骤二：匹配SMILES

1. **合并数据**：
   - 使用`pert_id`作为键，将`sig_info_df`和`small_molecules_df`中的数据合并，保留所有`sig_info_df`中的行，以及`small_molecules_df`中匹配的`canonical_smiles`列。
2. **保存结果**：
   - 将合并后的数据保存到`GSE92742_Broad_LINCS_sig_info_with_smiles.txt`文件中。

## 步骤三：筛选包含BRD的行

1. **加载数据**：
   - 从`GSE92742_Broad_LINCS_sig_info_with_smiles.txt`文件加载已添加SMILES的数据到`filtered_df`。
2. **筛选数据**：
   - 筛选出`pert_id`中包含"BRD"的行。
3. **保存结果**：
   - 将筛选后的数据保存到`GSE92742_Broad_LINCS_sig_info_filtered.txt`文件中。

## 步骤四：筛选有5次以上重复的分子

1. **加载数据**：
   - 从`GSE92742_Broad_LINCS_sig_info_filtered.txt`文件加载筛选后的数据到`replicated_df`。
2. **统计SMILES出现次数**：
   - 统计每个`canonical_smiles`的出现次数。
   - 筛选出出现次数大于5次的SMILES。
3. **筛选数据**：
   - 保留SMILES出现次数大于5次的行。
4. **保存结果**：
   - 将筛选后的数据保存到`GSE92742_Broad_LINCS_sig_info_replicated.txt`文件中。

## 步骤五：验证SMILES的有效性

1. **加载数据**：
   - 从`GSE92742_Broad_LINCS_sig_info_replicated.txt`文件加载数据到`validated_df`。
2. **验证SMILES**：
   - 使用RDKit库验证每个SMILES是否有效。
3. **筛选数据**：
   - 保留SMILES有效的行。
4. **保存结果**：
   - 将验证后的数据保存到`GSE92742_Broad_LINCS_sig_info_final.txt`文件中。

## 步骤六：统计基本信息

1. **加载数据**：
   - 从`GSE92742_Broad_LINCS_sig_info_final.txt`文件加载最终处理后的数据到`sig_info`。
2. **统计信息**：
   - 输出前五行数据。
   - 统计唯一`sig_id`、`pert_id`和`pert_iname`的数量。
   - 统计包含"BRD"的独一无二的`pert_id`数量。


In [2]:
import pandas as pd
from rdkit import Chem

# 文件路径配置
sig_info_path = '/mnt/d/Research/PHD/DLEPS/data/GSE92742/GSE92742_Broad_LINCS_sig_info.txt'
small_molecules_path = '/mnt/d/Research/PHD/DLEPS/data/GSE92742/LINCS_small_molecules.tsv'
output_with_smiles_path = '/mnt/d/Research/PHD/DLEPS/results/GSE92742_Broad_LINCS_sig_info_with_smiles.txt'
output_filtered_path = '/mnt/d/Research/PHD/DLEPS/results/GSE92742_Broad_LINCS_sig_info_filtered.txt'
output_replicated_path = '/mnt/d/Research/PHD/DLEPS/results/GSE92742_Broad_LINCS_sig_info_replicated.txt'
output_final_path = '/mnt/d/Research/PHD/DLEPS/results/GSE92742_Broad_LINCS_sig_info_final.txt'

In [3]:
def analyze_statistics(file_path):
    """统计文件中的基本信息"""
    df = pd.read_csv(file_path, sep="\t", dtype=str)
    
    # 总数量统计
    total_sig_id_count = df['sig_id'].count()
    total_pert_id_count = df['pert_id'].count()
    total_pert_iname_count = df['pert_iname'].count()
    total_canonical_smiles_count = df['canonical_smiles'].count()
    
    # 不重复数量统计
    unique_sig_id_count = df['sig_id'].nunique()
    unique_pert_id_count = df['pert_id'].nunique()
    unique_pert_iname_count = df['pert_iname'].nunique()
    unique_canonical_smiles_count = df['canonical_smiles'].nunique()
    
    # canonical_smiles为空和不为空的统计
    empty_smiles_count = df['canonical_smiles'].isna().sum()
    non_empty_smiles_count = df['canonical_smiles'].notna().sum()
    
    # pert_id 开头是 BRD 的统计
    brd_entries = df[df['pert_id'].str.startswith('BRD', na=False)]
    total_brd_count = brd_entries.shape[0]
    unique_brd_pert_id_count = brd_entries['pert_id'].nunique()
    empty_smiles_in_brd_count = brd_entries['canonical_smiles'].isna().sum()
    non_empty_smiles_in_brd_count = brd_entries['canonical_smiles'].notna().sum()
    
    # BRD 开头的条目中重复次数最多的前五个 pert_id
    top_brd_pert_ids = (
        brd_entries['pert_id']
        .value_counts()
        .head(5)
    )
    
    # pert_type 列的统计
    pert_type_counts = df['pert_type'].value_counts()
    total_pert_type_count = len(pert_type_counts)
    
    # 查找 canonical_smiles 相同但 pert_id 不同的数据
    duplicated_smiles = df[df.duplicated(subset=['canonical_smiles'], keep=False)]
    result = []
    for smile, group in duplicated_smiles.groupby('canonical_smiles'):
        if group['pert_id'].nunique() > 1:
            result.append(group)
    
    if result:
        result_df = pd.concat(result)
        result_file_path = '/mnt/d/Research/PHD/DLEPS/results/duplicated_smiles_diff_pertid.csv'
        result_df.to_csv(result_file_path, index=False)
        duplicated_smiles_message = f"数据已保存到文件: {result_file_path}"
    else:
        duplicated_smiles_message = "没有找到具有相同 canonical_smiles 但不同 pert_id 的数据。"

    # 打印统计信息
    print(f"文件: {file_path}")
    print(f"总数量统计:")
    print(f"  sig_id 总数: {total_sig_id_count}")
    print(f"  pert_id 总数: {total_pert_id_count}")
    print(f"  pert_iname 总数: {total_pert_iname_count}")
    print(f"  canonical_smiles 总数: {total_canonical_smiles_count}")
    print(f"不重复数量统计:")
    print(f"  不重复的 sig_id 数量: {unique_sig_id_count}")
    print(f"  不重复的 pert_id 数量: {unique_pert_id_count}")
    print(f"  不重复的 pert_iname 数量: {unique_pert_iname_count}")
    print(f"  不重复的 canonical_smiles 数量: {unique_canonical_smiles_count}")
    print(f"附加统计信息:")
    print(f"  具有空 canonical_smiles 的 pert_id 数量: {empty_smiles_count}")
    print(f"  具有非空 canonical_smiles 的 pert_id 数量: {non_empty_smiles_count}")
    print(f"pert_id 开头是 BRD 的统计:")
    print(f"  BRD 开头的条目总数: {total_brd_count}")
    print(f"  不重复的 BRD 开头的 pert_id 数量: {unique_brd_pert_id_count}")
    print(f"  BRD 开头的条目中，canonical_smiles 为空的数量: {empty_smiles_in_brd_count}")
    print(f"  BRD 开头的条目中，canonical_smiles 不为空的数量: {non_empty_smiles_in_brd_count}")
    print(f"  BRD 开头的条目中重复次数最多的前五个 pert_id 名称及其重复次数:")
    print(top_brd_pert_ids)
    print(f"pert_type 列的统计:")
    print(f"  不同种类的数量: {total_pert_type_count}")
    print(f"  每个种类的数量:")
    print(pert_type_counts)
    print(f"相同 canonical_smiles 但不同 pert_id 的数据检查:")
    print(duplicated_smiles_message)
    print("-" * 50)


In [ ]:
def add_smiles_column():
    """Step 1-3: 匹配 SMILES 并添加到数据"""
    sig_info_df = pd.read_csv(sig_info_path, sep='\t')
    small_molecules_df = pd.read_csv(small_molecules_path, sep='\t')
    merged_df = sig_info_df.merge(small_molecules_df[['pert_id', 'canonical_smiles']], on='pert_id', how='left')
    merged_df.to_csv(output_with_smiles_path, sep='\t', index=False)
    print(f"添加canonical_smiles列后的结果已保存到: {output_with_smiles_path}")
    analyze_statistics(output_with_smiles_path)  # 输出文件统计信息

add_smiles_column()


In [ ]:
def filter_brd_data():
    """Step 4-5: 筛选出包含 BRD 的行"""
    merged_df = pd.read_csv(output_with_smiles_path, sep='\t')
    filtered_df = merged_df[merged_df['pert_id'].str.contains('BRD', na=False)]
    filtered_df.to_csv(output_filtered_path, sep='\t', index=False)
    print(f"过滤后的结果（仅保留pert_id包含BRD）已保存到: {output_filtered_path}")
    analyze_statistics(output_filtered_path)  # 输出文件统计信息

filter_brd_data()


In [ ]:
def filter_replicates():
    """Step 6: 保留有5次以上重复的分子"""
    filtered_df = pd.read_csv(output_filtered_path, sep='\t')
    
    # 统计每个 SMILES 的出现次数
    smiles_counts = filtered_df['canonical_smiles'].value_counts()
    replicated_smiles = smiles_counts[smiles_counts > 5].index
    replicated_df = filtered_df[filtered_df['canonical_smiles'].isin(replicated_smiles)]
    
    replicated_df.to_csv(output_replicated_path, sep='\t', index=False)
    print(f"筛选后（SMILES 出现次数大于 5）的数据已保存到: {output_replicated_path}")
    analyze_statistics(output_replicated_path)  # 输出文件统计信息

filter_replicates()


In [ ]:
def validate_smiles():
    """Step 7-8: 验证 SMILES 是否有效"""
    replicated_df = pd.read_csv(output_replicated_path, sep='\t')
    
    # 验证 SMILES 是否能被 RDKit 解析
    def is_valid_smiles(smiles):
        try:
            mol = Chem.MolFromSmiles(smiles)
            return mol is not None
        except:
            return False
    
    validated_df = replicated_df[replicated_df['canonical_smiles'].apply(is_valid_smiles)]
    validated_df.to_csv(output_final_path, sep='\t', index=False)
    print(f"最终处理后的数据已保存到: {output_final_path}")
    analyze_statistics(output_final_path)  # 输出文件统计信息

validate_smiles()


In [ ]:
from cmapPy.pandasGEXpress import parse, write_gct

gctx_file = "/mnt/d/Research/PHD/DLEPS/References/LINCS L1000 Chemical Perturbations (2021).gctx"  # 替换为实际路径
data = parse.parse(gctx_file)

# 查看数据
print(data.data_df.head())

expression_data = data.data_df

# 示例：查看前几行
print(expression_data.head())

row_metadata = data.row_metadata_df
print(row_metadata.head())


col_metadata = data.col_metadata_df
print(col_metadata.head())


In [ ]:
import pandas as pd

def check_line_breaks(file_path, expected_columns):
    """检查文件中换行有问题的行"""
    with open(file_path, 'r') as file:
        lines = file.readlines()
    
    problematic_lines = []
    for i, line in enumerate(lines):
        columns = line.split('\t')
        if len(columns) != expected_columns:
            problematic_lines.append((i + 1, line))
    
    if problematic_lines:
        print(f"在文件 {file_path} 中发现换行有问题的行:")
        for line_num, content in problematic_lines:
            print(f"行号 {line_num}: {content}")
    else:
        print(f"文件 {file_path} 中没有发现换行有问题的行。")

# 假设文件中应该有 14 列（根据提供的示例数据格式）
expected_columns = 13
check_line_breaks('/mnt/d/Research/PHD/DLEPS/results/GSE92742_Broad_LINCS_sig_info_with_smiles.txt', expected_columns)